In [ ]:
import os
import warnings
import sys
import ipywidgets as widgets
import plotly.graph_objects as go
import numpy as np
import pandas as pd
from IPython.display import display

warnings.filterwarnings("ignore")
module_path = os.path.abspath(os.path.join("../../.."))

if module_path not in sys.path:
    sys.path.append(module_path)

from openbb_terminal.sdk import openbb

In [ ]:
z_dict = {
    "IV": "Implied Volatility",
    "OI": "Open Interest",
    "LP": "Last Price",
}
strings = {
    "IV": "DTE: %{x} <br>Strike: %{y} <br>IV: %{z}<extra></extra>",
    "OI": "DTE: %{x} <br>Strike: %{y} <br>OI: %{z}<extra></extra>",
    "LP": "DTE: %{x} <br>Strike: %{y} <br>LP: %{z}<extra></extra>",
}


def create_layout(fig, width, height, z, ticker):
    fig.update_layout(
        title=f"{z.upper()} Surface for {ticker.upper()}",
        autosize=False,
        width=width,
        height=height,
        scene=dict(
            xaxis_title="Days to Expiration",
            yaxis_title="Strike",
            zaxis_title=z_dict[z],
        ),
    )

In [ ]:
class Chart:
    def __init__(self):
        self.last_ticker = ""
        self.df = pd.DataFrame()

    def create(self, z, ticker, height, width):
        if ticker:
            if ticker != self.last_ticker:
                self.df = openbb.stocks.options.vsurf(ticker)
                self.last_ticker = ticker

            if not self.df.empty:
                z_dict = {
                    "IV": self.df.impliedVolatility,
                    "OI": self.df.openInterest,
                    "LP": self.df.lastPrice,
                }
                fig = go.Figure(
                    data=[
                        go.Mesh3d(
                            z=z_dict[z],
                            x=self.df.dte,
                            y=self.df.strike,
                            intensity=np.sqrt(z_dict[z]),
                            colorscale="Jet",
                            hovertemplate=strings[z],
                            showscale=False,
                        )
                    ]
                )

                create_layout(fig, height, width, z, ticker)
                if os.environ.get("SERVER_SOFTWARE", "jupyter").startswith("voila"):
                    fig.show(config={"showTips": False}, renderer="notebook")
                else:
                    fig.show(config={"showTips": False})


opts = ["IV", "OI", "LP"]
z_widget = widgets.Dropdown(options=opts, value="IV", description="Data")
tickers_widget = widgets.Text(description="Ticker", value="TSLA")
h_widget = widgets.IntText(value=1000, description="Height")
w_widget = widgets.IntText(value=1000, description="Width")

controls = widgets.VBox([tickers_widget, z_widget])
size = widgets.VBox([h_widget, w_widget])
controls = widgets.HBox([controls, size])

chart = Chart()
stocks_view = widgets.interactive_output(
    chart.create,
    {"z": z_widget, "ticker": tickers_widget, "height": h_widget, "width": w_widget},
)

title_html = "<h1>Option Surface Dashboard</h1>"
app_contents = [
    widgets.HTML(title_html),
    controls,
    stocks_view,
]
app = widgets.VBox(app_contents)
display(app)